<a href="https://colab.research.google.com/github/Eltonomwega/ML_Notebooks/blob/main/Group_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [186]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error

**Importing data**

In [119]:
data = pd.read_csv('/content/drive/MyDrive/4th year sem 1/Machine learning/data/Housing Pricing.csv')

In [120]:
data.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


In [121]:
data.drop('Id',axis=1,inplace=True)

In [122]:
#checking for any null values in the dataset
data.isna().any()

MSSubClass       False
MSZoning         False
LotFrontage       True
LotArea          False
Street           False
                 ...  
MoSold           False
YrSold           False
SaleType         False
SaleCondition    False
SalePrice        False
Length: 76, dtype: bool

### Preparing data

In [123]:
#features in the dataset
data.count()
original_features = list(data.columns)
original_features

['MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch

In [124]:
#list of features with missing values
features_missing = list(data.columns[data.isna().any()])
features_missing

['LotFrontage',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [125]:
#dropping features with missing value percentage >40%
data.dropna(axis=1,thresh=0.4*(len(data)),inplace=True)

In [126]:
data.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF'

In [127]:
mean_fill = ['LotFrontage','MasVnrArea']
bfill=['FireplaceQu']
ffill = list(set(features_missing)-set(mean_fill)-set(bfill))

In [128]:
#fill missing values in mean_fill with mean
for feature in mean_fill:
  data[feature].fillna(data[feature].mean(),inplace=True)

In [129]:
#fill missing values with forward fill
for feature in ffill:
  data[feature].ffill(inplace=True)

In [130]:
#fill missing values with backward fill
for feature in bfill:
  data[feature].bfill(inplace=True)

In [131]:
#check for null values
data.isna().any()

MSSubClass       False
MSZoning         False
LotFrontage      False
LotArea          False
Street           False
                 ...  
MoSold           False
YrSold           False
SaleType         False
SaleCondition    False
SalePrice        False
Length: 76, dtype: bool

### Encoding the dataset

In [132]:
# isolate all categorical fields in the dataset 
# by finding fields of datatype object
cat_data=data.select_dtypes(include=['object']).copy()
cat_data.head(2)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal


In [133]:
cat_features=list(cat_data.columns)
cat_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [134]:
# separate the categorical features into nominal, ordinal and numerical
nominal= ['MSZoning','LandContour','LotConfig','Neighborhood']
ordinal=list(set(cat_features)-set(nominal))
numerical=list(set(original_features)-set(cat_features))
target=['SalePrice']

In [135]:
data[nominal].head()

,MSZoning,LandContour,LotConfig,Neighborhood
0,RL,Lvl,Inside,CollgCr
1,RL,Lvl,FR2,Veenker
2,RL,Lvl,Inside,CollgCr
3,RL,Lvl,Corner,Crawfor
4,RL,Lvl,FR2,NoRidge


In [136]:
data[ordinal].head()

,ExterQual,Utilities,BsmtFinType2,HouseStyle,PavedDrive,RoofMatl,BsmtExposure,BsmtCond,Condition1,Functional,Street,LotShape,MasVnrType,GarageQual,Heating,KitchenQual,Exterior2nd,BsmtQual,HeatingQC,Exterior1st,GarageType,RoofStyle,BsmtFinType1,GarageFinish,GarageCond,SaleType,BldgType,CentralAir,Condition2,FireplaceQu,Foundation,ExterCond,SaleCondition,LandSlope,Electrical
0,Gd,AllPub,Unf,2Story,Y,CompShg,No,TA,Norm,Typ,Pave,Reg,BrkFace,TA,GasA,Gd,VinylSd,Gd,Ex,VinylSd,Attchd,Gable,GLQ,RFn,TA,WD,1Fam,Y,Norm,TA,PConc,TA,Normal,Gtl,SBrkr
1,TA,AllPub,Unf,1Story,Y,CompShg,Gd,TA,Feedr,Typ,Pave,Reg,None,TA,GasA,TA,MetalSd,Gd,Ex,MetalSd,Attchd,Gable,ALQ,RFn,TA,WD,1Fam,Y,Norm,TA,CBlock,TA,Normal,Gtl,SBrkr
2,Gd,AllPub,Unf,2Story,Y,CompShg,Mn,TA,Norm,Typ,Pave,IR1,BrkFace,TA,GasA,Gd,VinylSd,Gd,Ex,VinylSd,Attchd,Gable,GLQ,RFn,TA,WD,1Fam,Y,Norm,TA,PConc,TA,Normal,Gtl,SBrkr
3,TA,AllPub,Unf,2Story,Y,CompShg,No,Gd,Norm,Typ,Pave,IR1,None,TA,GasA,Gd,Wd Shng,TA,Gd,Wd Sdng,Detchd,Gable,ALQ,Unf,TA,WD,1Fam,Y,Norm,Gd,BrkTil,TA,Abnorml,Gtl,SBrkr
4,Gd,AllPub,Unf,2Story,Y,CompShg,Av,TA,Norm,Typ,Pave,IR1,BrkFace,TA,GasA,Gd,VinylSd,Gd,Ex,VinylSd,Attchd,Gable,GLQ,RFn,TA,WD,1Fam,Y,Norm,TA,PConc,TA,Normal,Gtl,SBrkr


In [137]:
# encode the ordinal data with numbers
for feature in ordinal:
  data[feature] = (data[feature].astype('category')).cat.codes

In [138]:
df_ordinal=data[ordinal]

In [139]:
# encode the nominal data using one hot encode
df_nominal=pd.get_dummies(data[nominal])

In [140]:
df_numerical=data[numerical]

In [141]:
# concat the datasets to form an encoded dataframe
encoded_data=pd.concat([df_numerical,df_nominal,df_ordinal],axis=1)

In [142]:
encoded_data.head()

,GarageYrBlt,MSSubClass,BedroomAbvGr,TotalBsmtSF,LotArea,YrSold,SalePrice,BsmtFinSF2,BsmtUnfSF,OverallQual,BsmtFullBath,MasVnrArea,PoolArea,KitchenAbvGr,LotFrontage,YearBuilt,HalfBath,2ndFlrSF,YearRemodAdd,1stFlrSF,LowQualFinSF,GarageCars,TotRmsAbvGrd,GarageArea,WoodDeckSF,3SsnPorch,BsmtFinSF1,MoSold,FullBath,Fireplaces,OpenPorchSF,MiscVal,BsmtHalfBath,GrLivArea,ScreenPorch,OverallCond,EnclosedPorch,MSZoning_C (all),MSZoning_FV,MSZoning_RH,...,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,ExterQual,Utilities,BsmtFinType2,HouseStyle,PavedDrive,RoofMatl,BsmtExposure,BsmtCond,Condition1,Functional,Street,LotShape,MasVnrType,GarageQual,Heating,KitchenQual,Exterior2nd,BsmtQual,HeatingQC,Exterior1st,GarageType,RoofStyle,BsmtFinType1,GarageFinish,GarageCond,SaleType,BldgType,CentralAir,Condition2,FireplaceQu,Foundation,ExterCond,SaleCondition,LandSlope,Electrical
0,2003.0,60,3,856,8450,2008,208500,0,150,7,1,196.0,0,1,65.0,2003,1,854,2003,856,0,2,8,548,0,0,706,2,2,0,61,0,0,1710,0,5,0,0,0,0,...,0,0,0,0,0,2,0,5,5,2,1,3,3,2,6,1,3,1,4,1,2,13,2,0,12,1,1,2,1,4,8,0,1,2,4,2,4,4,0,4
1,1976.0,20,3,1262,9600,2007,181500,0,284,6,0,0.0,0,1,80.0,1976,0,0,1976,1262,0,2,6,460,298,0,978,5,2,1,0,0,1,1262,0,8,0,0,0,0,...,0,0,0,0,1,3,0,5,2,2,1,1,3,1,6,1,3,2,4,1,3,8,2,0,8,1,1,0,1,4,8,0,1,2,4,1,4,4,0,4
2,2001.0,60,3,920,11250,2008,223500,0,434,7,1,162.0,0,1,68.0,2001,1,866,2002,920,0,2,6,608,0,0,486,9,2,1,42,0,0,1786,0,5,0,0,0,0,...,0,0,0,0,0,2,0,5,5,2,1,2,3,2,6,1,0,1,4,1,2,13,2,0,12,1,1,2,1,4,8,0,1,2,4,2,4,4,0,4
3,1998.0,70,3,756,9550,2006,140000,0,540,7,1,0.0,0,1,60.0,1915,0,756,1970,961,0,3,7,642,0,0,216,2,1,1,35,0,0,1717,0,5,272,0,0,0,...,0,0,0,0,0,3,0,5,5,2,1,3,1,2,6,1,0,2,4,1,2,15,3,2,13,5,1,0,2,4,8,0,1,2,2,0,4,0,0,4
4,2000.0,60,4,1145,14260,2008,250000,0,490,8,1,350.0,0,1,84.0,2000,1,1053,2000,1145,0,3,9,836,192,0,655,12,2,1,84,0,0,2198,0,5,0,0,0,0,...,0,0,0,0,0,2,0,5,5,2,1,0,3,2,6,1,0,1,4,1,2,13,2,0,12,1,1,2,1,4,8,0,1,2,4,2,4,4,0,4


In [143]:
encoded_data.shape

(1460, 111)

### Data Normalization

In [144]:
#isolate the x in the dataset
X=encoded_data.drop('SalePrice',axis=1)

In [145]:
y=data[target]

In [146]:
#convert the dataframes to a numpy array
X=X.to_numpy()
y=y.to_numpy()

In [147]:
# standardize the data
X=StandardScaler().fit_transform(X)
y=StandardScaler().fit_transform(y)

In [148]:
X

array([[ 0.98587121,  0.07337496,  0.16377912, ...,  0.2085023 ,
        -0.22571613,  0.30240343],
       [-0.10254651, -0.87256276,  0.16377912, ...,  0.2085023 ,
        -0.22571613,  0.30240343],
       [ 0.90524768,  0.07337496,  0.16377912, ...,  0.2085023 ,
        -0.22571613,  0.30240343],
       ...,
       [-1.51345838,  0.30985939,  1.39002276, ...,  0.2085023 ,
        -0.22571613,  0.30240343],
       [-1.15065247, -0.87256276, -1.06246453, ...,  0.2085023 ,
        -0.22571613, -3.50370869],
       [-0.54597595, -0.87256276,  0.16377912, ...,  0.2085023 ,
        -0.22571613,  0.30240343]])

In [149]:
y

array([[ 0.34727322],
       [ 0.00728832],
       [ 0.53615372],
       ...,
       [ 1.07761115],
       [-0.48852299],
       [-0.42084081]])

### Feature Selection using L1

In [150]:
#split the dataset 


# assign Lasso
reg = LassoCV()

In [151]:
reg.fit(X,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [152]:
reg.coef_

array([ 0.00000000e+00, -6.60286822e-02, -9.87040470e-03,  3.35373060e-02,
        4.61709517e-02, -7.21247046e-03,  6.91942955e-03, -0.00000000e+00,
        1.67649003e-01,  3.57256943e-02,  2.90547332e-02, -0.00000000e+00,
       -1.64139913e-02, -2.40164482e-02,  5.16835084e-02,  9.62535771e-04,
        0.00000000e+00,  1.42945879e-02,  2.08182232e-03, -3.84955025e-04,
        8.63145247e-02,  4.62794380e-02,  0.00000000e+00,  2.02458333e-02,
        5.34748356e-03,  4.34693525e-02, -8.24371223e-03,  3.24309082e-02,
        3.48815680e-02,  2.01678259e-04, -4.05519092e-05,  1.46937036e-03,
        2.66550766e-01,  3.00932542e-02,  5.83051400e-02, -0.00000000e+00,
       -1.93738135e-02,  6.46353200e-03, -0.00000000e+00,  0.00000000e+00,
       -1.48239634e-03, -3.61908283e-02,  1.74321354e-02, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  2.08698268e-02, -1.82005949e-02,
       -1.21491061e-02,  0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        1.62334480e-04,  

In [153]:
# getting feature importance by getting the absolute value
importance = np.abs(reg.coef_)

In [154]:
importance

array([0.00000000e+00, 6.60286822e-02, 9.87040470e-03, 3.35373060e-02,
       4.61709517e-02, 7.21247046e-03, 6.91942955e-03, 0.00000000e+00,
       1.67649003e-01, 3.57256943e-02, 2.90547332e-02, 0.00000000e+00,
       1.64139913e-02, 2.40164482e-02, 5.16835084e-02, 9.62535771e-04,
       0.00000000e+00, 1.42945879e-02, 2.08182232e-03, 3.84955025e-04,
       8.63145247e-02, 4.62794380e-02, 0.00000000e+00, 2.02458333e-02,
       5.34748356e-03, 4.34693525e-02, 8.24371223e-03, 3.24309082e-02,
       3.48815680e-02, 2.01678259e-04, 4.05519092e-05, 1.46937036e-03,
       2.66550766e-01, 3.00932542e-02, 5.83051400e-02, 0.00000000e+00,
       1.93738135e-02, 6.46353200e-03, 0.00000000e+00, 0.00000000e+00,
       1.48239634e-03, 3.61908283e-02, 1.74321354e-02, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.08698268e-02, 1.82005949e-02,
       1.21491061e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.62334480e-04, 3.36667322e-03, 9.32187458e-04, 0.00000000e+00,
      

### Deep Feed Forward ANN

In [174]:
# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [180]:
#training the model
epochs=200

model_2 = Sequential()
model_2.add(Dense(4, activation='tanh'))
model_2.add(Dense(2, activation='tanh'))
model_2.add(Dense(1, activation='sigmoid'))

model_2.compile(Adam(learning_rate=0.01), 'binary_crossentropy', metrics=['Precision','Recall'])
history = model_2.fit(X_train, y_train, verbose=0, epochs=epochs)

In [181]:
model_2.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 4)                 444       
_________________________________________________________________
dense_28 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 3         
Total params: 457
Trainable params: 457
Non-trainable params: 0
_________________________________________________________________


### Test the data

In [187]:
y_hat = model_2.predict(X_test)

In [198]:
historydf=pd.DataFrame(history.history, index=history.epoch)
historydf.head()

,loss,precision,recall
0,-0.034623,1.0,0.415851
1,-0.929657,1.0,0.295499
2,-1.441611,1.0,0.253425
3,-1.867778,1.0,0.236791
4,-2.256756,1.0,0.240705


In [199]:
historydf['F-Measure']=(2*historydf['precision']*historydf['recall'])/(historydf['precision']+historydf['recall'])

In [200]:
historydf.head()

,loss,precision,recall,F-Measure
0,-0.034623,1.0,0.415851,0.587422
1,-0.929657,1.0,0.295499,0.456193
2,-1.441611,1.0,0.253425,0.404372
3,-1.867778,1.0,0.236791,0.382911
4,-2.256756,1.0,0.240705,0.388013


In [201]:
# Accuracy score
mean_absolute_error(y_test, y_hat)

0.5934919474144876